In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json

statesofInterest = ['IOWA', 'INDIANA', 'ILLINOIS', 'KANSAS', 'MINNESOTA', 'MISSOURI', 'NEBRASKA', 'NORTH DAKOTA', 'SOUTH DAKOTA', 'OHIO']

In [8]:
def crop_yield(year_param):

     # These are the parameters in the API that you choose.
    ploads = {'format': 'JSON',
              'source_desc': 'SURVEY',
              'sector_desc': 'CROPS',
              'frequency_desc': 'YEAR',
              'group_desc': 'FIELD CROPS',
              'commodity_desc': 'SOYBEANS',
              'short_desc': 'SOYBEANS - ACRES PLANTED', 
              #'prodn_practice_desc': 'ALL PRODUCTION PRACTICES',
              #'prodn_practice_desc': 'SOYBEANS, NON-IRRIGATED-ACRES PLANTED',
              #'statisticcat_desc': 'YIELD',
              'agg_level_desc': 'STATE',
              'state_name': statesofInterest,
              'year__GT': year_param,
              #'unit_desc': 'ACRES HARVESTED'
             }
              
    # key (make a parameter)
    r = requests.get("https://quickstats.nass.usda.gov/api/api_GET/?key=1C757E50-5169-30CC-BEFD-40A5C3E2A43D", params=ploads)
    # execute get request
    dt1 = json.dumps(r.json(), indent=2, sort_keys=True)
    # make it json format
    obj1 = json.loads(dt1)

    # collect relevant data for each object created
    dt2 = []
    for i in obj1["data"]:
        dt2.append([i["year"], i["reference_period_desc"], i["commodity_desc"], i["Value"], i['unit_desc'], i['state_name'], i['prodn_practice_desc']] )
        #dt2.append([i])
    # convert from dictionary format to pandas
    dt3 = pd.DataFrame.from_dict(dt2)
    return dt3

In [9]:
ts = crop_yield('1800')
#ts.to_csv("tmp.csv")
ts.head(10)

,0,1,2,3,4,5,6
0,2020,YEAR,SOYBEANS,"10,400,000",ACRES,ILLINOIS,ALL PRODUCTION PRACTICES
1,2020,YEAR - JUN ACREAGE,SOYBEANS,"10,400,000",ACRES,ILLINOIS,ALL PRODUCTION PRACTICES
2,2020,YEAR - MAR ACREAGE,SOYBEANS,"10,500,000",ACRES,ILLINOIS,ALL PRODUCTION PRACTICES
3,2019,YEAR,SOYBEANS,"9,950,000",ACRES,ILLINOIS,ALL PRODUCTION PRACTICES
4,2019,YEAR - AUG ACREAGE,SOYBEANS,"10,000,000",ACRES,ILLINOIS,ALL PRODUCTION PRACTICES
5,2019,YEAR - JUN ACREAGE,SOYBEANS,"10,300,000",ACRES,ILLINOIS,ALL PRODUCTION PRACTICES
6,2019,YEAR - MAR ACREAGE,SOYBEANS,"10,500,000",ACRES,ILLINOIS,ALL PRODUCTION PRACTICES
7,2019,YEAR - OCT ACREAGE,SOYBEANS,"10,000,000",ACRES,ILLINOIS,ALL PRODUCTION PRACTICES
8,2018,YEAR,SOYBEANS,"10,800,000",ACRES,ILLINOIS,ALL PRODUCTION PRACTICES
9,2018,YEAR - JUN ACREAGE,SOYBEANS,"10,900,000",ACRES,ILLINOIS,ALL PRODUCTION PRACTICES


In [10]:
ts.columns = ['Year', 'Forecast', 'Crop', 'Value', 'Unit', 'State', 'Product_Practices']
#ts['Value'] = ts['Value'].str.replace('NA', '').astype(int)

ts['Value'] = ts['Value'].str.strip()
ts['Value'] = ts['Value'].replace('(NA)', '0')
ts['Value'] = ts['Value'].str.replace(',', '').astype(int)

ts.sort_values('Year', inplace=True)
ts=ts.loc[ts['Forecast'] == "YEAR"]
ts.set_index('Year', inplace=True)
ts["Value"]=ts["Value"].astype("int")

ts.sort_values(["State", "Year"], inplace=True)
ts

,Forecast,Crop,Value,Unit,State,Product_Practices
Year,,,,,,
1924,YEAR,SOYBEANS,315000,ACRES,ILLINOIS,ALL PRODUCTION PRACTICES
1925,YEAR,SOYBEANS,290000,ACRES,ILLINOIS,ALL PRODUCTION PRACTICES
1926,YEAR,SOYBEANS,350000,ACRES,ILLINOIS,ALL PRODUCTION PRACTICES
1927,YEAR,SOYBEANS,429000,ACRES,ILLINOIS,ALL PRODUCTION PRACTICES
1928,YEAR,SOYBEANS,463000,ACRES,ILLINOIS,ALL PRODUCTION PRACTICES
...,...,...,...,...,...,...
2016,YEAR,SOYBEANS,5200000,ACRES,SOUTH DAKOTA,ALL PRODUCTION PRACTICES
2017,YEAR,SOYBEANS,5650000,ACRES,SOUTH DAKOTA,ALL PRODUCTION PRACTICES
2018,YEAR,SOYBEANS,5650000,ACRES,SOUTH DAKOTA,ALL PRODUCTION PRACTICES


In [11]:
iowa1 = ts[ts["State"] == "IOWA"]
mich1 = ts[ts["State"] == "MINNESOTA"]
nebr1 = ts[ts["State"] == "NEBRASKA"]

state1= pd.concat([iowa1["Value"], mich1["Value"], nebr1["Value"]], axis=1).sort_index()
state1.columns = ["Iowa", "Minnesota", "Nebraska"]
state1.to_csv("SoybeanStatesAcres.csv")